## Considerações Iniciais

Treinar modelos que envolvam técnicas de Deep Learning e detecção de imagens geralmente necessitam de uma grande quantia de dados para realizar predições precisas e construir modelos robustos.

É importante também frisarmos que, para os requisitos propostos queremos realizar a tarefa de **Detecção de Objetos**, que difere da tarefa de **Classificação de Imagens** uma vez que essa segunda não se preocupa em encontrar o objeto na imagem (por meio de *bounding box* por exemplo), mas sim informar que imagem é de determinada classe.

Um algoritmo bastante utilizado para tarefas de detecção de objeto em imagens é o algoritmo YOLO, este utiliza da lógica de encadeamento de CNNs (Redes Neurais Convolucionais) para realizar tarefas de visão computacional. Além disso o YOLO é um modelo já treinado em uma enorme base de imagens, fazendo com que ele generalize muito bem a tarefa de detecção.

Com as informações ditas acima, para o objetivo desejado podemos utilizar o modelo YOLO e reaproveitar seus pesos, realizando um processo de Fine-tuning, ou seja iremos ajustá-lo ao nosso dataset de imagens de fumaça repassado. A utilização do YOLO é bem simples e amigável com o pacote `ultralytics`, e a sua versão mais nova é a versão 11, que será utilizada para este projeto.



In [12]:
import cv2
from ultralytics import YOLO
import matplotlib.pyplot as plt
import numpy as np

## Coleta das Imagens



Com a utilização do YOLO em mente, precisamos de um dataset que possua imagens e suas correspondências de fumaça/fogo em bounding box, iniciei a busca de imagens que possam se aproximar das desejadas na biblioteca de imagens https://universe.roboflow.com/. 

O objetivo na coleção era encontrar um dataset de imagens possivelmente tiradas por câmeras em torres, como não temos essa informação fácil, escolhemos um dataset que:
- A fumaça/fogo não estivesse tão próximo da câmera;
- Pudessemos traçar uma linha do horizonte definida;
- Sem imagens de drone, satélite ou demais vistas aéreas.
- Neste primeiro momento evitei também datasets com muitas imagens direcionadas para a chama.

Foi utilizada as palavras chaves: *wildfire smoke detection forest fire*. 

Um dataset que atendeu as especificações foi o:
https://universe.roboflow.com/tristandeeplearnnig/wildfire-smoke-detection-v7bqc

É possível realizar o download do dataset utilizando o snippet de código abaixo, porém para garantir a disponibilização e evitar a necessidade de cadastro para recuperar a API key, ele também foi baixado em zip e salvo no repositório Git (apenas para caráter de projeto, dados de produção podem ser sensíveis e não devem ser salvos em Git).

In [2]:
# !pip install roboflow

# from roboflow import Roboflow
# rf = Roboflow(api_key="F8D7SH4AFY4XXgmh1F2h")
# project = rf.workspace("tristandeeplearnnig").project("wildfire-smoke-detection-v7bqc")
# version = project.version(1)
# dataset = version.download("yolov11")
                

Observações: Por utilizar um dataset baixado de terceiro, a não ser que verifiquemos imagem por imagem não podemos garantir a qualidade da rotulação e criação dos bounding boxes, além de garantir a total falta de Data Leakege por exemplo. A biblioteca YOLO não apresenta funções nativas para verificar em grande escala essa designação. Na próxima seção irei verificar o perfil do bounding box de uma pequena amostra das imagens baixadas. 

Temos como quantia:
- Treino: 1.378
- Validação: 147
- Teste: 74

O dataset apresenta apenas uma classe: Smoke

## Verificar o perfil das imagens

Irei realizar uma verificação manual do perfil da rotulagem e das imagens, para isso irei gerar as imagens com as informações de Bounding Box em uma pasta a parte, onde podemos visualizar se as Bounding Box criadas estão correspondente a fumaça.

Para manter este notebook enxuto, criei as funções auxiliares para ler o arquivo YAML de caminho dos dados e criar as imagens com as bounding box nos arquivos `src/utils.py` e `src/image_evaluation.py` respectivamente.

In [3]:
from src.utils import read_yaml
from src.image_evaluation import iterate_images_and_labels

images_with_bb_path = "output/evaluation_images"
path_file = read_yaml("data/data.yaml")

labels = iterate_images_and_labels(path_file["train"], images_with_bb_path)

Arquivo yaml: data/data.yaml carregado com sucesso.
Imagens para avaliação visual criadas.


Com as imagens geradas na pasta `output/evaluation_images` foi possível perceber algumas características do conjunto de dados utilizado:

- Há a presença de muitas imagens repetidas, por exemplo 2 imagens idênticas ou quase idênticas para o mesmo foco de fumaça;
- As imagens já estão no tamanho 640x640, tamanho esperado para aplicação ao YOLO;
- As imagens já apresentam determinado flip horizontal, esta poderia ser uma etapa do pré-processamento por exemplo, uma vez que pegamos a imagem e invertemos horizontalmente para que o modelo possa assimilar características da fumaça tanto crescente para o lado direito como para o esquerdo. Este comportamento levanta a dúvida se essa etapa de pré-processamento segue necessária.

Além disso os labels contidos nos arquivos de rótulo YOLO foram salvos para serem retornados.

In [15]:
# Exemplo de array com objetos (dtype=object)
labels = np.array(labels, dtype=object)
none_elements = labels[labels == None]
none_elements

array([], dtype=object)

Vemos que todas as imagens apresentam uma Bounding Box, ou seja, não há imagens que nos traga a informação apenas de background, em outras condições pode ser interessante adicionar imagens que não possuam o objeto a ser detectado para um melhor treinamento do modelo.

In [ ]:
model = YOLO("yolov8n.pt")  # load a pretrained model (recommended for training)

model.train(data="among.yaml", epochs=30, device=0)  # train the model
metrics = model.val()  # evaluate model performance on the validation set
results = model("https://ultralytics.com/images/bus.jpg")  # predict on an image

## Pré-processamento


TODO: Pesquisar melhores etapas de pré-processamento para fumaça

Treinamento do Modelo

NameError: name 'results' is not defined